In [1]:
import datetime
import os
import pandas as pd
from typing import Sequence

In [2]:
def read_partition(path: str) -> pd.DataFrame:
    def _read_partition(path: str, partition_cols: Sequence[Sequence[str]]) -> pd.DataFrame:
        if path.lower().endswith(".xlsx"):
            partitions = dict(partition_cols)
            if f"device{partitions["Device"]}" not in path:
                raise Exception(f"File {path} in wrong Device directory")
            df = pd.read_excel(path)
            return [ df.assign(**partitions) ]
        elif os.path.isdir(path):
            return [
                df
                for subpath in os.listdir(path)
                for df in _read_partition(f"{path}/{subpath}", partition_cols + ([subpath.split("=")] if "=" in subpath else []))
            ]
        else:
            return []
    
    dfs = _read_partition(path, [])
    return pd.concat(dfs) if dfs else pd.DataFrame()


def parse_tamtop_datetime(input: str) -> datetime.datetime:
    return datetime.datetime.fromisoformat(
        input.replace(" ", "T").replace("(", "").replace(")", "")
    )

In [3]:
df = read_partition("data")
df["Time"] = df["Time"].apply(parse_tamtop_datetime)
df = df.set_index(["Device", "Time"]).drop(["NO."], axis=1)
display(df)

PM2.5(ug/m³)  AQI  CO₂(ppm)  TVOC  \
Device Time                                                           
02     2024-08-21 04:00:00+08:00           5.2   29       593    51   
       2024-08-21 05:00:00+08:00           4.5   25       617    63   
       2024-08-21 06:00:00+08:00           4.4   25       647    53   
03     2024-08-17 20:00:00+08:00          47.8  131       580    28   
       2024-08-17 21:00:00+08:00          61.1  155       493    67   
...                                        ...  ...       ...   ...   
       2024-08-20 11:00:00+08:00           9.1   51       610    57   
       2024-08-20 12:00:00+08:00           5.9   33       647    44   
       2024-08-21 04:00:00+08:00           7.5   42       598    50   
       2024-08-21 05:00:00+08:00           6.8   38       604    88   
       2024-08-21 06:00:00+08:00           6.8   38       609    89   

                                  Temperature(℉)  Humidity(%RH) Floor  \
Device Time                                                             
02     2024-08-21 04:00:00+08:00            79.9             58     2   
       2024-08-21 05:00:00+08:00            79.7             58     2   
       2024-08-21 06:00:00+08:00            80.1             57     2   
03     2024-08-17 20:00:00+08:00            77.7             57     2   
       2024-08-17 21:00:00+08:00            81.9             51     2   
...                                          ...            ...   ...   
       2024-08-20 11:00:00+08:00            76.5             55     2   
       2024-08-20 12:00:00+08:00            75.6             53     2   
       2024-08-21 04:00:00+08:00            78.1             66     2   
       2024-08-21 05:00:00+08:00            78.3             65     2   
       2024-08-21 06:00:00+08:00            78.3             64     2   

                                      Location  
Device Time                                     
02     2024-08-21 04:00:00+08:00  West Hallway  
       2024-08-21 05:00:00+08:00  West Hallway  
       2024-08-21 06:00:00+08:00  West Hallway  
03     2024-08-17 20:00:00+08:00  West Hallway  
       2024-08-17 21:00:00+08:00  West Hallway  
...                                        ...  
       2024-08-20 11:00:00+08:00      Unit 210  
       2024-08-20 12:00:00+08:00      Unit 210  
       2024-08-21 04:00:00+08:00      Unit 210  
       2024-08-21 05:00:00+08:00      Unit 210  
       2024-08-21 06:00:00+08:00      Unit 210  

[184 rows x 8 columns]